### Задачи к Лекции 3

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("../data/mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,1,0,0,1,0,0,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,0,0,1,1,0,0,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,0,0,1,1,0,0,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,1,0,0,1,0,0,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,1,0,0,1,0,0,0


## Классы в Python

Нередко, возникает необходимость создания объектов с каким-нибудь внутренним поведением и состоянием. Примерами таких объектов являются классификаторы sklearn, массивы numpy и много другое. Такой объект можно объявить с помощью ключевого слова **class**

```python
class SomeObject:
    def __init__(self, depth):
        self.a = depth
        self.target = None
        
    def fit(self, data, target):
        self.target = data
        # magic
        return 
    
    def predict(self, data):
        return self.target    
```

После этого в коде можно будет создать экземпляр данного класса
```python
a = SomeObject(depth=5)
a.fit(data, target)
a.predict(data)
```

## Задачи

**1. В sklearn на данный момент отсутствует функционал для построения деревьев решений из категориальных данных, поэтому его нужно сделать самостоятельно и проверить его работу. Что нужно сделать:**

* __создать классификатор используя только pandas, numpy и scipy. Необходимо его сделать самому, используя исключительно только numpy, pandas и scipy (запрещено использовать sklearn и прочие библиотеки). Напоминаю, что для категориальных данных операция < или > не имеют смысла (использовать только != и ==). Гиперпараметром данного классификатора должна быть максимальная глубина дерева.__
* __Проверить работу данного классификатора на наборе ("gender", "cholesterol", "gluc").__
* __С помощью кросс-валидации найти оптимальную глубину этого дерева. Для вашего классификатора GridSearchCV не подойдет, придется это сделать также самостоятельно.__
* __Нарисовать полученное дерево (я должен понять, как и откуда вы его нарисовали).__

Алгоритм работы классификатора:
 1. Перебираем все возможные признаки и смотрим либо неопределенность Джини, либо прирост информации. Это даст критерий разбиения в виде "признак == значение"
 2. Если выборка полученная при разбиении состоит из объектов одного класса (соответсвует нулевой энтропии), то данный лист просто возвращает значение этого класса.
 3. В противном случае, образуется новый узел и для него начинаем с пункта 1.
 4. Если достигли максимальной глубины, то вместа узла создаем лист, который возвращает самое вероятное значение.
 
__Замечание:__ в этой задаче не нужно использовать onehot-кодирование.

In [4]:
from sklearn.model_selection import train_test_split
data = df[["gender", "cholesterol", "gluc"]].values
#data = df[["gender", "cholesterol", "gluc", "smoke", "alco", "active"]], df['cardio']
target = df['cardio'].values
train, test, target_train, target_test = train_test_split(   
    data, target, 
    test_size=0.3)

In [3]:
class RecursiveTree:
    def __init__(self, max_depth): 
        self.max_depth = max_depth

        # Есть всегда (листьев и узлов)
        self.p0 = None
        self.p1 = None
        self.size = None
        self.target = None
        self.entropy = None
        
        # Есть только у узлов
        self.feature_num = None
        self.feature_value = None
        self.childs = None
        
    def set_depth(self, max_depth):
        self.max_depth = max_depth
        if self.childs is not None:
            self.childs[0].set_depth(max_depth - 1)
            self.childs[1].set_depth(max_depth - 1)
    
    def _entropy(self, values):
        """
            Считаем энтропию для вектора целевых исходов
        """
        p = values.sum() / values.shape[0]
        q = 1.0 - p        
        return - np.nan_to_num(p * np.log2(p)) - np.nan_to_num(q * np.log2(q))
    
    def print(self, names, tab=0):              
        if self.childs is not None:
            print("  "*tab, "[%4s == %2d][%6d]" % (names[self.feature_num], self.feature_value, self.size), 
              "%4.2f %4.2f %2d %7.5f" %(self.p0, self.p1, self.target, self.entropy))
            self.childs[0].print(names, tab+1)
            self.childs[1].print(names, tab+1)
        else:
            print("  "*tab, "[____ == __][%6d]" % self.size, 
              "%4.2f %4.2f %2d %7.5f" %(self.p0, self.p1, self.target, self.entropy))
            
    def _get_dot_code(self, names, name, parent=None):
        """
            names - имена признаков, для расшифровки
            name - имя узла (уникальное)
            parent - имя родительского узла
        """
        content = "\n"

        if self.childs is not None:
            content += '%s [label="%s == %s\\nS = %.3f\\nsamples = %d\\nprob = [%.2f, %.2f]", fillcolor="#%X"];\n' % (
                name, 
                names[self.feature_num], self.feature_value, 
                self.entropy, self.size, 
                self.p0, self.p1,
                (0xe5813900 if self.p0 > self.p1 else 0x399de500) 
                + int(0xff * (self.p0 if self.p0 > self.p1 else self.p1))
            )               
        else:
            content += '%s [label="S = %.3f\\nsamples = %d\\nprob = [%.2f, %.2f]", fillcolor="#%X"];\n' % (
                name, 
                self.entropy, self.size, 
                self.p0, self.p1,
                (0xe5813900 if self.p0 > self.p1 else 0x399de500) 
                + int(0xff * (self.p0 if self.p0 > self.p1 else self.p1))
            )               
            
        if parent is not None:
            content += "%s -> %s;" % (parent, name)
            
        if self.childs is not None:
            content += self.childs[0]._get_dot_code(names, name + "f", name)
            content += self.childs[1]._get_dot_code(names, name + "t", name)
            
        return content
            
    def to_dot(self, filename, names):
        f = open(filename, "w")
        content = self._get_dot_code(names, "root", None)
        f.write("digraph Tree {\n")
        f.write('\tnode [shape=box, style="filled", color="black"];\n')
        f.write(content)
        f.write("}")
        f.close()        
        
    def _predict_proba(self, features):
        if self.childs is None or self.max_depth <= 0:
            return self.p0, self.p1
        
        if features[self.feature_num] == self.feature_value:
            return self.childs[1]._predict_proba(features)
        else:
            return self.childs[0]._predict_proba(features)
        
    def _predict(self, features):
        if self.childs is None or self.max_depth <= 0:
            return self.target
        
        if features[self.feature_num] == self.feature_value:
            return self.childs[1]._predict(features)
        else:
            return self.childs[0]._predict(features)
        
    def predict(self, data):
        data = np.array(data)
        result = np.zeros(data.shape[0])
        for i, features in enumerate(data):
            result[i] = self._predict(features)
        return result
    
    def predict_proba(self, data):
        data = np.array(data)
        result = np.zeros( (data.shape[0], 2) )
        for i, features in enumerate(data):
            result[i] = self._predict_proba(features)
        return result
    
    def fit(self, data, target):
        data = np.array(data)
        target = np.array(target)
        
        # Заполняем данные о текущем дереве
        mask = target == 0
        self.size = target.shape[0]
        self.p0 = mask.sum() / self.size
        self.p1 = 1.0 - self.p0
        self.target = 1 if self.p1 > self.p0 else 0
        self.entropy = self._entropy(target)        
        
        if self.entropy == 0:
            return
                
        # Строим разбиение        
        n_features = data.shape[1]        
        features = [np.unique(data[:,i]) for i in range(n_features)]
        
        split_best = None
        split_feature = None
        split_feature_value = None
        split_mask_true = None
        split_mask_false = None
        
        for i, feature in enumerate(features):
            # Если признак состоит из одного значения, то по нему нельзя сделать разбиение
            if len(feature) < 2:
                continue
                
            for fv in feature:
                mask = data[:, i] == fv
                not_mask = np.logical_not(mask)
                
                S_true = self._entropy(target[mask])
                S_false = self._entropy(target[not_mask])
                
                p_true = mask.sum() / mask.shape[0]
                p_false = 1.0 - p_true
                
                dS = p_true * S_true + p_false * S_false
                
                if split_best is None or split_best > dS:
                    split_best = dS
                    split_feature = i
                    split_feature_value = fv
                    split_mask_true = mask
                    split_mask_false = not_mask

        if split_best is None:
            return
                   
        self.feature_num = split_feature
        self.feature_value = split_feature_value

        self.childs = [RecursiveTree(self.max_depth - 1), RecursiveTree(self.max_depth - 1)]
        self.childs[0].fit(data[split_mask_false], target[split_mask_false])
        self.childs[1].fit(data[split_mask_true], target[split_mask_true])

In [13]:
from sklearn.metrics import accuracy_score

tree = RecursiveTree(3)
tree.fit(train, target_train)
pv = tree.predict_proba(test)
yv = tree.predict(test)

print("Accuracy = ", accuracy_score(target_test, yv))

tree.to_dot("graph.dot", names=["gend", "chol", 'gluc', 'smok', 'alco', 'actv'])

Accuracy =  0.5931428571428572


In [14]:
!dot -Tpng "graph.dot" -o "graph.png"

<img src="graph.png"/>

**Комментарии:** Ваши комментарии здесь.